<a id="neo0"></a>

# SAE 502 : Migration et optimisation de la base Comptoir de SQL à Neo4j  


**Auteurs :** Enora Aveline, Rose Nerriec, Ariane Souchard


<a id="neo0"></a>

# 1.H2GIS

In [1]:
#%pip show shapely h2gis

In [2]:
#%pip install shapely h2gis

In [3]:
from h2gis import H2GIS
import pandas as pd

h2gis = H2GIS("D:\\comptoir_h2gis")

In [4]:
h2gis.fetch("show tables")

{'TABLE_NAME': ['CATEGORIE',
  'CLIENT',
  'COMMANDE',
  'DETAILCOMMANDE',
  'EMPLOYE',
  'FOURNISSEUR',
  'GEOMETRY_COLUMNS',
  'MESSAGER',
  'PRODUIT',
  'SPATIAL_REF_SYS'],
 'TABLE_SCHEMA': ['PUBLIC',
  'PUBLIC',
  'PUBLIC',
  'PUBLIC',
  'PUBLIC',
  'PUBLIC',
  'PUBLIC',
  'PUBLIC',
  'PUBLIC',
  'PUBLIC']}

### Question 1 - Recréation de la contrainte `EMPLOYE_FK`

Lors de la migration vers H2GIS, la contrainte de clé étrangère `EMPLOYE_FK` a été supprimée
pour permettre l’insertion des employés dans un ordre quelconque.

On souhaite maintenant **la recréer**.  
Il s’agit d’une contrainte de clé étrangère **auto-référencée** entre :

- `EMPLOYE.RENDCOMPTEA` (le supérieur hiérarchique)
- `EMPLOYE.NOEMP` (l’identifiant de l’employé)

La contrainte est recréée via le calepin Jupyter à l’aide de `h2gis.execute`.


In [5]:
res = h2gis.execute("""
ALTER TABLE EMPLOYE
ADD CONSTRAINT EMPLOYE_FK
FOREIGN KEY (RENDCOMPTEA)
REFERENCES EMPLOYE(NOEMP);
""")
print("Résultat de l'ALTER TABLE :", res)

Résultat de l'ALTER TABLE : -1


### Question 2 - Diagramme relationnel de la base *Comptoir* (DBeaver)

Le schéma relationnel de la base *Comptoir* généré avec DBeaver est présenté ci-dessous :

![Diagramme relationnel de la base Comptoir](diagramme_relationnel.png)


### Question 3 – Requêtes sans `*` avec restitution dans un DataFrame pandas

On souhaite remplacer les requêtes du type :

`SELECT * FROM NOM_TABLE ORDER BY COLONNE;`

par des requêtes **sans `*`**, en écrivant explicitement les colonnes de chaque table.
    
L’objectif est d’obtenir le **même résultat** qu’avec un `SELECT * ... ORDER BY ...`,  
mais en respectant la bonne pratique qui consiste à **ne pas utiliser `*`** dans les requêtes SQL.

In [6]:
#CLIENT 
query_client = """
SELECT 
    CODECLI, SOCIETE, CONTACT, FONCTION, ADRESSE, 
    VILLE, REGION, CODEPOSTAL, PAYS, TEL, FAX
FROM CLIENT
ORDER BY CODECLI
"""

data_client = h2gis.fetch(query_client)
df_client = pd.DataFrame(data_client)
df_client

,CODECLI,SOCIETE,CONTACT,FONCTION,ADRESSE,VILLE,REGION,CODEPOSTAL,PAYS,TEL,FAX
0,ALFKI,Alfreds Futterkiste,Maria Anders,ReprÃ©sentant(e),Obere Str. 57,Berlin,,12209,Allemagne,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,PropriÃ©taire,Avda. de la ConstituciÃ³n 2222,MÃ©xico D.F.,,5021,Mexique,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno TaquerÃ­a,Antonio Moreno,PropriÃ©taire,Mataderos 2312,MÃ©xico D.F.,,5023,Mexique,(5) 555-3932,
3,AROUT,Around the Horn,Thomas Hardy,ReprÃ©sentant(e),120 Hanover Sq.,London,,WA1 1DP,Royaume-Uni,(71) 555-7788,(71) 555-6750
4,BERGS,Berglunds snabbkÃ¶p,Christina Berglund,Acheteur,BerguvsvÃ¤gen 8,LuleÃ¥,,S-958 22,SuÃ¨de,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...
89,WARTH,Wartian Herkku,Pirkko Koskitalo,Chef comptable,Torikatu 38,Oulu,,90110,Finlande,981-443655,981-443655
90,WELLI,Wellington Importadora,Paula Parente,Chef des ventes,"Rua do Mercado, 12",Resende,SP,08737-363,BrÃ©sil,(14) 555-8122,
91,WHITC,White Clover Markets,Karl Jablonski,PropriÃ©taire,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,Etats-Unis,(206) 555-4112,(206) 555-4115
92,WILMK,Wilman Kala,Matti Karttunen,Assistant du marketing,Keskuskatu 45,Helsinki,,21240,Finlande,90-224 8858,90-224 8858


In [7]:
# EMPLOYE
query_employe = """
SELECT 
    NOEMP,
    NOM,
    PRENOM,
    FONCTION,
    TITRECOURTOISIE,
    ADRESSE,
    VILLE,
    REGION,
    CODEPOSTAL,
    PAYS,
    TELDOM,
    EXTENSION,
    RENDCOMPTEA,
    DATENAISSANCE,
    DATEEMBAUCHE
FROM EMPLOYE
ORDER BY NOEMP
"""
data_employe = h2gis.fetch(query_employe)
df_employe = pd.DataFrame(data_employe)
df_employe


,NOEMP,NOM,PRENOM,FONCTION,TITRECOURTOISIE,ADRESSE,VILLE,REGION,CODEPOSTAL,PAYS,TELDOM,EXTENSION,RENDCOMPTEA,DATENAISSANCE,DATEEMBAUCHE
0,1,Davolio,Nancy,ReprÃ©sentant(e),Mlle,507 - 20th Ave. E. Apt. 2A,Seattle,WA,98122,Etats-Unis,(206) 555-9857,5467,2,1978-12-08,2012-05-01
1,2,Fuller,Andrew,Vice-PrÃ©sident,Dr.,908 W. Capital Way,Tacoma,WA,98401,Etats-Unis,(206) 555-9482,3457,0,1982-02-19,2012-08-14
2,3,Leverling,Janet,ReprÃ©sentant(e),Mlle,722 Moss Bay Blvd.,Kirkland,WA,98033,Etats-Unis,(206) 555-3412,3355,2,1993-08-30,2012-04-01
3,4,Peacock,Margaret,ReprÃ©sentant(e),Mme,4110 Old Redmond Rd.,Redmond,WA,98052,Etats-Unis,(206) 555-8122,5176,2,1967-09-19,2013-05-03
4,5,Buchanan,Steven,Chef des ventes,M.,14 Garrett Hill,London,,SW1 8JR,Royaume-Uni,(71) 555-4848,3453,2,1985-03-04,2013-10-17
5,6,Suyama,Michael,ReprÃ©sentant(e),M.,Coventry House Miner Rd.,London,,EC2 7JR,Royaume-Uni,(71) 555-7773,428,7,1993-07-02,2013-10-17
6,7,Emery,Patrick,Chef des ventes,M.,Edgeham Hollow Winchester Way,London,,RG1 9SP,Royaume-Uni,(71) 555-5598,465,5,1990-05-29,2014-01-02
7,8,Callahan,Laura,Assistante commerciale,Mlle,4726 - 11th Ave. N.E.,Seattle,WA,98105,Etats-Unis,(206) 555-1189,2344,2,1988-01-09,2014-03-05
8,9,Dodsworth,Anne,ReprÃ©sentant(e),Mlle,7 Houndstooth Rd.,London,,WG2 7LT,Royaume-Uni,(71) 555-4444,452,5,1996-01-27,2014-11-15
9,10,Suyama,Jordan,ReprÃ©sentant(e),M.,Coventry House Miner Rd.,London,,EC2 7JR,Royaume-Uni,(71) 555-7773,428,7,1993-07-02,2013-10-21


In [8]:
# COMMANDE
query_commande = """
SELECT 
    NOCOM, 
    CODECLI, 
    NOEMP, 
    DATECOM, 
    ALIVAVANT, 
    DATEENV, 
    NOMESS, 
    PORT, 
    DESTINATAIRE, 
    ADRLIV, 
    VILLELIV, 
    REGIONLIV, 
    CODEPOSTALLIV, 
    PAYSLIV 
FROM COMMANDE 
ORDER BY NOCOM
"""

data_commande = h2gis.fetch(query_commande)
df_commande = pd.DataFrame(data_commande)
df_commande



,NOCOM,CODECLI,NOEMP,DATECOM,ALIVAVANT,DATEENV,NOMESS,PORT,DESTINATAIRE,ADRLIV,VILLELIV,REGIONLIV,CODEPOSTALLIV,PAYSLIV
0,10248,VINET,5,2016-08-04,2016-09-01,2016-08-16,3,162,Vins et alcools Chevalier,59 rue de lAbbaye,Reims,,51100,France
1,10249,TOMSP,6,2016-08-05,2016-09-16,2016-08-10,1,58,Toms SpezialitÃ¤ten,Luisenstr. 48,MÃ¼nster,,44087,Allemagne
2,10250,HANAR,4,2016-08-08,2016-09-05,2016-08-12,2,329,Hanari Carnes,"Rua do PaÃ§o, 67",Rio de Janeiro,RJ,05454-876,BrÃ©sil
3,10251,VICTE,3,2016-08-08,2016-09-05,2016-08-15,1,207,Victuailles en stock,"2, rue du Commerce",Lyon,,69004,France
4,10252,SUPRD,4,2016-08-09,2016-09-06,2016-08-11,2,256,SuprÃªmes dÃ©lices,"Boulevard Tirou, 255",Charleroi,,B-6000,Belgique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,2018-06-04,2018-07-02,,2,125,Pericles Comidas clÃ¡sicas,Calle Dr. Jorge Cash 321,MÃ©xico D.F.,,5033,Mexique
826,11074,SIMOB,7,2018-06-05,2018-07-03,,2,92,Simons bistro,VinbÃ¦ltet 34,KÃ¸benhavn,,1734,Danemark
827,11075,RICSU,8,2018-06-05,2018-07-03,,2,31,Richter Supermarkt,Starenweg 5,GenÃ¨ve,,1204,Suisse
828,11076,BONAP,4,2018-06-05,2018-07-03,,2,191,Bon app,"12, rue des Bouchers",Marseille,,13008,France


In [9]:
#DETAILCOMMANDE
query_detailcommande = """
SELECT 
    NOCOM, 
    REFPROD, 
    QTE, 
    PRIXUNIT, 
    REMISE 
FROM DETAILCOMMANDE 
ORDER BY NOCOM, REFPROD
"""

data_detailcommande = h2gis.fetch(query_detailcommande)
df_detailcommande = pd.DataFrame(data_detailcommande)
df_detailcommande

,NOCOM,REFPROD,QTE,PRIXUNIT,REMISE
0,10248,11,12,70.0,0.00
1,10248,42,10,49.0,0.00
2,10248,72,5,174.0,0.00
3,10249,14,9,93.0,0.00
4,10249,51,40,212.0,0.00
...,...,...,...,...,...
2150,11077,64,2,166.0,0.03
2151,11077,66,1,85.0,0.00
2152,11077,73,2,75.0,0.01
2153,11077,75,4,39.0,0.00


In [10]:
# PRODUIT
query_produit = """
SELECT 
    REFPROD, 
    NOMPROD, 
    NOFOUR, 
    CODECATEG, 
    QTEPARUNIT, 
    PRIXUNIT, 
    UNITESSTOCK,
    UNITESCOM,
    NIVEAUREAP, 
    INDISPONIBLE 
FROM PRODUIT 
ORDER BY REFPROD
"""

data_produit = h2gis.fetch(query_produit)
df_produit = pd.DataFrame(data_produit)
df_produit


,REFPROD,NOMPROD,NOFOUR,CODECATEG,QTEPARUNIT,PRIXUNIT,UNITESSTOCK,UNITESCOM,NIVEAUREAP,INDISPONIBLE
0,1,Chai,1,1,10 boÃ®tes x 20 sacs,90.00,39,0,10,False
1,2,Chang,1,1,24 bouteilles (1 litre),95.00,17,40,25,False
2,3,Aniseed Syrup,1,2,12 bouteilles (550 ml),50.00,13,70,25,False
3,4,Chef Antons Cajun Seasoning,2,2,48 pots (6 onces),110.00,53,0,0,False
4,5,Chef Antons Gumbo Mix,2,2,36 boÃ®tes,106.75,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...
79,80,Calvados,18,1,1 bouteille (750 cc),150.00,50,20,10,False
80,81,PavÃ© dIsygny,30,4,15 unitÃ©s (300 g),250.00,30,10,0,False
81,82,Calvados de Mutrecy,30,1,1 bouteille (750 cc),145.00,30,0,10,False
82,83,PoirÃ©,30,1,12 bouteilles (75 cl),300.00,40,5,10,False


In [11]:
# CATEGORIE
query_categorie = """
SELECT 
    CODECATEG, 
    NOMCATEG, 
    DESCRIPTION 
FROM CATEGORIE 
ORDER BY CODECATEG
"""

data_categorie = h2gis.fetch(query_categorie)
df_categorie = pd.DataFrame(data_categorie)
df_categorie


,CODECATEG,NOMCATEG,DESCRIPTION
0,1,Boissons,"Boissons, cafÃ©s, thÃ©s, biÃ¨res"
1,2,Condiments,"Sauces, assaisonnements et Ã©pices"
2,3,Desserts,Desserts et friandises
3,4,Produits laitiers,Fromages
4,5,PÃ¢tes et cÃ©rÃ©ales,"Pains, biscuits, pÃ¢tes et cÃ©rÃ©ales"
5,6,Viandes,Viandes prÃ©parÃ©es
6,7,Produits secs,"Fruits secs, raisins, autres"
7,8,Poissons et fruits de mer,"Poissons, fruits de mer, escargots"


In [12]:
# FOURNISSEUR
query_fournisseur = """
SELECT 
    NOFOUR, 
    SOCIETE, 
    CONTACT, 
    FONCTION, 
    ADRESSE, 
    VILLE, 
    REGION, 
    CODEPOSTAL, 
    PAYS, 
    TEL, 
    FAX, 
    PAGEACCUEIL 
FROM FOURNISSEUR 
ORDER BY NOFOUR
"""

data_fournisseur = h2gis.fetch(query_fournisseur)
df_fournisseur = pd.DataFrame(data_fournisseur)
df_fournisseur


,NOFOUR,SOCIETE,CONTACT,FONCTION,ADRESSE,VILLE,REGION,CODEPOSTAL,PAYS,TEL,FAX,PAGEACCUEIL
0,1,Exotic Liquids,Charlotte Cooper,Assistant export,49 Gilbert St.,London,,EC1 4SD,Royaume-Uni,(171) 555-2222,,
1,2,New Orleans Cajun Delights,Shelley Burke,Acheteur,P.O. Box 78934,New Orleans,LA,70117,Etats-Unis,(100) 555-4822,,Cajun.htm#CAJUN.HTM#
2,3,Grandma Kellys Homestead,Regina Murphy,ReprÃ©sentant(e),707 Oxford Rd.,Ann Arbor,MI,48104,Etats-Unis,(313) 555-5735,(313) 555-3349,
3,4,Tokyo Traders,Yoshi Nagase,Directeur du marketing,9-8 Sekimai Musashino-shi,Tokyo,,100,Japon,(03) 3555-5011,,
4,5,Cooperativa de Quesos Las Cabras,Antonio del Valle Saavedra,Responsable export,Calle del Rosal 4,Oviedo,Asturias,33007,Espagne,(98) 598 76 54,,
5,6,Mayumis,Mayumi Ohno,Chef de produit,92 Setsuko Chuo-ku,Osaka,,545,Japon,(06) 431-7877,,Mayumis (sur le World Wide Web)#http://www.mic...
6,7,"Pavlova, Ltd.",Ian Devling,Directeur du marketing,74 Rose St. Moonie Ponds,Melbourne,Victoria,3058,Australie,(059) 55-5450,(03) 444-6588,
7,8,"Specialty Biscuits, Ltd.",Peter Wilson,ReprÃ©sentant(e),29 Kings Way,Manchester,,M14 GSD,Royaume-Uni,(161) 555-4448,,
8,9,PB KnÃ¤ckebrÃ¶d AB,Lars Peterson,Assistant(e) des ventes,Kaloadagatan 13,GÃ¶teborg,,S-345 67,SuÃ¨de,031-987 65 43,031-987 65 91,
9,10,Refrescos Americanas LTDA,Carlos Diaz,Directeur du marketing,Av. das Americanas 12.890,SÃ£o Paulo,,5442,BrÃ©sil,(11) 555 4640,,


In [13]:
# MESSAGER
query_messager = """
SELECT 
    NOMESS, 
    NOMMESS, 
    TEL 
FROM MESSAGER 
ORDER BY NOMESS
"""

data_messager = h2gis.fetch(query_messager)
df_messager = pd.DataFrame(data_messager)
df_messager


,NOMESS,NOMMESS,TEL
0,1,Speedy Express,(503) 555-9831
1,2,United Package,(503) 555-3199
2,3,Federal Shipping,(503) 555-9931


### Question 4 – Modèle de données du graphe avec l’outil en ligne

Dans cette question, il s’agit de proposer le **modèle de données graphe** correspondant au cas Comptoir
en utilisant un logiciel de modélisation en ligne (par exemple Arrows.app ou un outil équivalent).

Les consignes de style à respecter sont les suivantes :

- Les **étiquettes de nœuds** (labels) doivent être au **singulier** et en **Upper Camel Case**  
  (par exemple : `Client`, `Commande`, `Produit`, `Employe`, `Fournisseur`, `Messager`, `Categorie`).

- Les **types de relations** doivent être écrits en **majuscules**, éventuellement avec des underscores `_`  
  (par exemple : `A_PASSE`, `CONTIENT`, `FOURNIT`, `TRAITE`, `EXPEDIE`, `GROUPE`, `SUPERVISE`).

- Les **clés des propriétés** doivent être en **lower camel case**  
  (par exemple : `codeCli`, `noEmp`, `dateCom`, `refProd`, `prixUnit`, `dateNaissance`, `dateEmbauche`).

Comme les noms des attributs dans H2/H2GIS sont en majuscules, ils ne sont **pas repris tels quels** dans le graphe :
nous les adaptons pour respecter les conventions de nommage de Neo4j.

Le modèle de données du graphe sera construit dans l’outil en ligne puis **exporté en image** (![modele donnes de la base Comptoir](modeles_donnees.png) 
et inséré dans le calepin pour illustrer la structure du graphe (nœuds, relations, propriétés).


### Question 5 – Ordre Cypher du modèle de données et commentaires sur le typage

Dans cette question, on doit incorporer dans le calepin l’**ordre Cypher** correspondant au modèle de données
du graphe (par exemple un `CREATE` schématique qui décrit les nœuds, les relations et les propriétés avec leurs types).

L’ordre Cypher pourra être corrigé par rapport à la version initiale obtenue depuis l’outil en ligne
afin de respecter :

- les **conventions de nommage** (labels, types de relations, clés de propriétés),
- les **types de données** choisis dans Neo4j  
  (par exemple : `INTEGER` pour les identifiants, `STRING` pour les textes, `BOOLEAN` pour les drapeaux,
  `DATE` pour les dates, etc.).

Les commentaires attendus portent notamment sur :

- les **types choisis** pour les propriétés des nœuds et des relations (par exemple pourquoi `idProjet` est un entier,
  pourquoi `dateCom` est convertie en type `date()` dans Cypher, etc.) ;
- le **typage dans Neo4j** par rapport au modèle relationnel d’origine (ce


In [14]:
h2gis.execute("""
DROP TABLE IF EXISTS CLIENT CASCADE CONSTRAINTS;
CREATE TABLE Client (
    codecli VARCHAR(50) PRIMARY KEY,
    societe VARCHAR(100),
    contact VARCHAR(100),
    fonction VARCHAR(50),
    adresse VARCHAR(200),
    ville VARCHAR(50),
    region VARCHAR(50),
    codepostal VARCHAR(10),
    pays VARCHAR(50),
    tel VARCHAR(20),
    fax VARCHAR(20)
);

DROP TABLE IF EXISTS COMMANDE CASCADE CONSTRAINTS;
CREATE TABLE Commande (
    nocom INT PRIMARY KEY AUTO_INCREMENT,
    codecli VARCHAR(10),
    noemp INT,
    nomess INT,
    datecom DATE,
    aliavant DATE,
    dateenv DATE,
    port VARCHAR(50),
    destinataire VARCHAR(100),
    adrliv VARCHAR(200),
    villeliv VARCHAR(50),
    regionliv VARCHAR(50),
    codepostalliv VARCHAR(10),
    paysliv VARCHAR(50),
    FOREIGN KEY (codecli) REFERENCES Client(codecli),
    FOREIGN KEY (noemp) REFERENCES Employe(noemp),
    FOREIGN KEY (nomess) REFERENCES Messager(nomess)
);

DROP TABLE IF EXISTS PRODUIT CASCADE CONSTRAINTS;
CREATE TABLE Produit (
    refprod INT PRIMARY KEY AUTO_INCREMENT,
    noFour INT,
    codeCateg INT,
    nomprod VARCHAR(100),
    qteparunit VARCHAR(50),
    prixunit INT,
    unitesstock INT,
    unitescom INT,
    niveaureap INT,
    indisponible BOOLEAN,
    FOREIGN KEY (nofour) REFERENCES Fournisseur(nofour),
    FOREIGN KEY (codecateg) REFERENCES Categorie(codecateg)
);

DROP TABLE IF EXISTS CATEGORIE CASCADE CONSTRAINTS;
CREATE TABLE Catégorie (
    codecateg INT PRIMARY KEY,
    nomcateg VARCHAR(100),
    description TEXT
);

DROP TABLE IF EXISTS EMPLOYE CASCADE CONSTRAINTS;
CREATE TABLE Employe (
    noemp INT PRIMARY KEY AUTO_INCREMENT,
    nom VARCHAR(50),
    prenom VARCHAR(50),
    fonction VARCHAR(50),
    titrecourtoisie VARCHAR(20),
    addresse VARCHAR(200),
    ville VARCHAR(50),
    region VARCHAR(50),
    codepostal VARCHAR(10),
    pays VARCHAR(50),
    teldom VARCHAR(20),
    extension VARCHAR(10),
    rendcomptea INT,
    datenaissance DATE,
    dateembauche DATE,
    chef INT,
    FOREIGN KEY (rendcomptea) REFERENCES Employe(noemp)
    
);

DROP TABLE IF EXISTS MESSAGER CASCADE CONSTRAINTS;
CREATE TABLE Messager (
    nomess INT PRIMARY KEY,
    nommess VARCHAR(50),
    tel VARCHAR(20)
);

DROP TABLE IF EXISTS FOURNISSEUR CASCADE CONSTRAINTS;
CREATE TABLE Fournisseur (
    nofour INT PRIMARY KEY AUTO_INCREMENT,
    societe VARCHAR(100),
    contact VARCHAR(100),
    fonction VARCHAR(50),
    adresse VARCHAR(200),
    ville VARCHAR(50),
    region VARCHAR(50),
    codepostal VARCHAR(10),
    pays VARCHAR(50),
    tel VARCHAR(20),
    fax VARCHAR(20),
    pageaccueil VARCHAR(100)
);
""")

-1

### Question 6 – Squelettes des ordres de migration vers Neo4j

On dérive, à partir des `CREATE TABLE` de la base H2GIS, des **squelettes d’ordres Cypher** qui décrivent :

- les nœuds (labels, propriétés),
- les relations (types, propriétés éventuelles),

mais **sans valeurs concrètes** (pas encore de paramètres).

Ces squelettes doivent être **cohérents** avec le modèle de données graphe proposé à la Question 4,  
et servent de base pour la future migration.



In [15]:
# 1. Création des Nœuds Fournisseur
h2gis.execute("""
CREATE (:Fournisseur {
    nofour: 0,
    societe: "",
    contact: "",
    fonction: "",
    adresse: "",
    ville: "",
    region: "",
    codepostal: "",
    pays: "",
    tel: "",
    fax: "",
    pageaccueil: ""
});
""")

# 2. Création des Nœuds Categorie
h2gis.execute("""
CREATE (:Categorie {
    codecateg: 0,
    nocateg: "",
    description: ""
});
""")


# 3. Création des Nœuds Produit
h2gis.execute("""
CREATE (:Produit {
    refprod: 0,
    nomprod: "",
    nofour: 0,
    codecateg: 0,
    qteparunit: "",
    prixunit: 0.0,
    unitesstock: 0,
    unitescom: 0,
    niveureap: 0,
    indisponible: FALSE
});
""")

# 4. Création des Nœuds Messager
h2gis.execute("""
CREATE (:Messager {
    nomess: 0,
    nommess: "",
    tel: ""
});
""")


# 5. Création des Nœuds Client
h2gis.execute("""
CREATE (:Client {
    codecli: "",
    societe: "",
    contact: "",
    fonction: "",
    adresse: "",
    ville: "",
    region: "",
    codePostal: "",
    pays: "",
    tel: "",
    fax: ""
});
""")


# 6. Création des Nœuds Employe
h2gis.execute("""
CREATE (:Employe {
    noemp: 0,
    nom: "",
    prenom: "",
    fonction: "",
    titrecourtoisie: "",
    datenaissance: date(),
    dateembauche: date(),
    adresse: "",
    ville: "",
    region: "",
    codepostal: "",
    pays: "",
    teldom: "",
    extension: "",
    rendcomptea: 0
});
""")

# 7. Création des Nœuds Commande
h2gis.execute("""
CREATE (:Commande {
    nocom: 0,
    codecli: "",
    noemp: 0,
    nomess: 0,
    datecom: date(),
    aLivavant: date(),
    dateenv: date(),
    adrliv: "",
    villeliv: "",
    regionliv: "",
    codepostalliv: "",
    paysliv: "",
    port: 0.0,
    destinataire: ""
});
""")

-1

### Question 7 – Ordres Cypher paramétrés pour une migration industrielle

À partir des squelettes obtenus à la Question 6, on construit des **ordres Cypher paramétrés** :

- les propriétés sont alimentées via des paramètres (`$codeCli`, `$noEmp`, `$dateCom`, `$refProd`, …),
- les relations (ex. `A_PASSE`, `CONTIENT`) peuvent aussi porter des paramètres (`$qte`, `$prixUnit`, `$remise`, …).

Ces ordres sont prévus pour être exécutés depuis Python dans une migration de **qualité industrielle**  
(sécurisée, réutilisable, adaptée à de gros volumes de données).



In [16]:
# Nœud Fournisseur
h2gis.execute("""
CREATE (:Fournisseur {
    nofour: $nofour,
    societe: $societe,
    contact: $contact,
    fonction: $fonction,
    adresse: $adresse,
    ville: $ville,
    region: $region,
    codepostal: $codepostal,
    pays: $pays,
    tel: $tel,
    fax: $fax,
    pageaccueil: $pageaccueil
});
""")
h2gis.execute("""
CREATE CONSTRAINT IF NOT EXISTS FOR (f:Fournisseur) REQUIRE f.nofour IS UNIQUE;
""")

# Nœud Categorie
h2gis.execute("""
CREATE (:Categorie {
    codecateg: $codecateg,
    nomcateg: $nomcateg,
    description: $description
});
""")
h2gis.execute("""
CREATE CONSTRAINT IF NOT EXISTS FOR (c:Categorie) REQUIRE c.codecateg IS UNIQUE;
""")




# Nœud Produit
h2gis.execute("""
CREATE (:Produit {
    refprod: $refprod,
    nomprod: $nomprod,
    nofour: $nofour,
    codecateg: $codecateg,
    qteparunit: $qteparunit,
    prixunit: $prixunit,
    unitesstock: $unitesstock,
    unitescom: $unitescom,
    niveureap: $niveureap,
    indisponible: $indisponible
});
""")
h2gis.execute("""
CREATE CONSTRAINT IF NOT EXISTS FOR (p:Produit) REQUIRE p.refprod IS UNIQUE;
""")


# Nœud Messager
h2gis.execute("""
CREATE (:Messager {
    nomess: $nomess,
    nommess: $nommess,
    tel: $tel
});
""")
h2gis.execute("""
CREATE CONSTRAINT IF NOT EXISTS FOR (m:Messager) REQUIRE m.nomess IS UNIQUE;
""")

# Nœud Client
h2gis.execute("""
CREATE (:Client {
    codecli: $codecli,
    societe: $societe,
    contact: $contact,
    fonction: $fonction,
    adresse: $adresse,
    ville: $ville,
    region: $region,
    codepostal: $codepostal,
    pays: $pays,
    tel: $tel,
    fax: $fax
});
""")
h2gis.execute("""
CREATE CONSTRAINT IF NOT EXISTS FOR (cl:Client) REQUIRE cl.codecli IS UNIQUE;
""")



# Nœud Employe
h2gis.execute("""
CREATE (:Employe {
    noemp: $noemp,
    nom: $nom,
    prenom: $prenom,
    fonction: $fonction,
    titrecourtoisie: $titrecourtoisie,
    datenaissance: date($datenaissance),
    dateembauche: date($dateembauche),
    adresse: $adresse,
    ville: $ville,
    region: $region,
    codepostal: $codepostal,
    pays: $pays,
    teldom: $teldom,
    extension: $extension,
    rendcomptea: $rendcomptea
});
""")
h2gis.execute("""
CREATE CONSTRAINT IF NOT EXISTS FOR (e:Employe) REQUIRE e.noemp IS UNIQUE;
""")

# Nœud Commande
h2gis.execute("""
CREATE (:Commande {
    nocom: $nocom,
    codecli: $codecli,
    noemp: $noemp,
    nomess: $nomess,
    datecom: date($datecom),
    alivavant: date($alivavant),
    dateenv: date($dateenv),
    adrliv: $adrliv,
    villeliv: $villeliv,
    regionliv: $regionliv,
    codePostalliv: $codepostalliv,
    paysliv: $paysliv,
    port: $port,
    destinataire: $destinataire
});
""")
h2gis.execute("""
CREATE CONSTRAINT IF NOT EXISTS FOR (co:Commande) REQUIRE co.nocom IS UNIQUE;
""")

# 1. Relations de Produit (Categoriser et Fournit_Par)
# Note : Nécessite que Categorie et Fournisseur existent
h2gis.execute("""
MATCH (c:Categorie {codecateg: $codecateg}), 
      (f:Fournisseur {nofour: $nofour}),
      (p:Produit {codecateg: $codecateg, nofour: $nofour, refprod: $refprod})
CREATE (c)-[:CATEGORISER]->(p),
       (f)-[:FOURNIT_PAR]->(p);
""")

# 2. Relations de Commande (A_Commander, A_Realiser, A_Livrer)
# Note : Nécessite que Client, Employe et Messager existent
h2gis.execute("""
MATCH (cl:Client {codecli: $codecli}), 
      (e:Employe {noemp: $noemp}), 
      (m:Messager {nomess: $nomess}),
      (co:Commande {codecli: $codecli, noemp: $noemp, nomess: $nomess, nocom: $nocom})
CREATE (cl)-[:A_COMMANDER]->(co),
       (e)-[:A_REALISER]->(co),
       (m)-[:A_LIVRER]->(co);
""")

# 3. Relation Récursive Employe (Rend_Compte_A)
# Note : Nécessite que les deux nœuds Employe existent
h2gis.execute("""
MATCH (employe:Employe {noemp: $noemp}), 
      (manager:Employe {noemp: $rendcomptea})
WHERE employe.rendcomptea = manager.noemp
CREATE (employe)-[:REND_COMPTE_A]->(manager);
""")

# 4. Relation de Jointure Commande-Produit (COMMANDE_DETAIL)
# Note : Nécessite que les nœuds Commande et Produit existent
h2gis.execute("""
MATCH (p:Produit {refprod: $refprod}), 
      (co:Commande {nocom: $nocom})
CREATE (p)-[:COMMANDE_DETAIL {
    qte: $qte,
    prixUnit: $prixunitdetail,
    remise: $remise
}]->(co);
""")

-1

### Question 8 – Choix du nom de la base Neo4j de destination
Nous avons choisi comme nom de base de données **comptoir**.


In [17]:
comptoir = h2gis

### Question 9 – Contraintes, index et procédures d’initialisation / vidage

Cette question demande de :

- **proposer des contraintes de clé de nœud** pour gérer les **doublons naturels**  
  (ex. unicité de `codeCli` pour `:Client`, de `refProd` pour `:Produit`, etc.),
- **proposer des index** sur les propriétés souvent utilisées en recherche (`nom`, `nomProd`, `dateCom`, …),
- coder, en Python, des **procédures utilitaires** qui :
  - vident le graphe (suppression des nœuds et relations),
  - initialisent le graphe (création des contraintes, index, et éventuellement données minimales de test).

Ces procédures s’appuient sur le pilote Neo4j pour Python et une fonction de type `query(...)`/`execute_statements_list(...)`.

In [18]:
h2gis.execute("""
CREATE CONSTRAINT client_pk IF NOT EXISTS FOR (c:Client) REQUIRE c.codecli IS UNIQUE;
CREATE CONSTRAINT employe_pk IF NOT EXISTS FOR (e:Employe) REQUIRE e.noemp IS UNIQUE;
CREATE CONSTRAINT messager_pk IF NOT EXISTS FOR (m:Messager) REQUIRE m.nomess IS UNIQUE;
CREATE CONSTRAINT fournisseur_pk IF NOT EXISTS FOR (f:Fournisseur) REQUIRE f.nofour IS UNIQUE;
CREATE CONSTRAINT categorie_pk IF NOT EXISTS FOR (c:Categorie) REQUIRE c.codecateg IS UNIQUE;
CREATE CONSTRAINT produit_pk IF NOT EXISTS FOR (p:Produit) REQUIRE p.refprod IS UNIQUE;
CREATE CONSTRAINT commande_pk IF NOT EXISTS FOR (c:Commande) REQUIRE c.nocom IS UNIQUE;
""")

-1

In [19]:
# EMPLOYE

h2gis.execute("""
CREATE INDEX employe_nom IF NOT EXISTS FOR (e:Employe) ON (e.nom);
CREATE INDEX employe_fonction IF NOT EXISTS FOR (e:Employe) ON (e.fonction);
CREATE INDEX employe_ville IF NOT EXISTS FOR (e:Employe) ON (e.ville);
""")


# CLIENT

h2gis.execute("""
CREATE INDEX client_societe IF NOT EXISTS FOR (c:Client) ON (c.societe);
CREATE INDEX client_ville IF NOT EXISTS FOR (c:Client) ON (c.ville);
CREATE INDEX client_pays IF NOT EXISTS FOR (c:Client) ON (c.pays);
""")

# FOURNISSEUR

h2gis.execute("""
CREATE INDEX fournisseur_societe IF NOT EXISTS FOR (f:Fournisseur) ON (f.societe);
CREATE INDEX fournisseur_ville IF NOT EXISTS FOR (f:Fournisseur) ON (f.ville);
CREATE INDEX fournisseur_pays IF NOT EXISTS FOR (f:Fournisseur) ON (f.pays);
""")

# COMMANDE

h2gis.execute("""
CREATE INDEX commande_date IF NOT EXISTS FOR (c:Commande) ON (c.dateCom);
CREATE INDEX commande_codeCli IF NOT EXISTS FOR (c:Commande) ON (c.codeCli);
CREATE INDEX commande_noEmp IF NOT EXISTS FOR (c:Commande) ON (c.noEmp);
""")



# PRODUIT

h2gis.execute("""
CREATE INDEX produit_nom IF NOT EXISTS FOR (p:Produit) ON (p.nomProd);
CREATE INDEX produit_indisponible IF NOT EXISTS FOR (p:Produit) ON (p.indisponible);
CREATE INDEX produit_codeCateg IF NOT EXISTS FOR (p:Produit) ON (p.codeCateg);
""")



# CATEGORIE

h2gis.execute("""
CREATE INDEX categorie_nom IF NOT EXISTS FOR (c:Categorie) ON (c.nomCateg);
""")

-1

In [20]:
import logging
import os
import neo4j
from neo4j import GraphDatabase, basic_auth
from json import dumps
from textwrap import dedent
from typing import cast


from typing_extensions import LiteralString
# Fixation du niveau de journalisation
logging.root.setLevel(logging.INFO)
logging.root.setLevel(logging.DEBUG)

In [21]:
os.environ['NEO4J_URI'] = 'localhost:7687'
os.environ['NEO4J_USER'] = 'neo4j'
os.environ['NEO4J_PASSWORD'] = 'password'
os.environ['NEO4J_SCHEME'] = 'neo4j' 
os.environ['NEO4J_DATABASE'] = 'comptoir'
os.environ['NEO4J_VERSION'] = '5'
os.environ['NEO4J_QUERY_STRING'] = ''

In [22]:
# Lecture des variables d'environnement concernant Neo4j
scheme_destination = os.getenv("NEO4J_SCHEME", "neo4j")
url_destination = os.getenv("NEO4J_URI", "localhost:7687")
username_destination = os.getenv("NEO4J_USER", "neo4j")
password_destination = os.getenv("NEO4J_PASSWORD", "password")
neo4j_version_destination = os.getenv("NEO4J_VERSION", "4")
database_destination = os.getenv("NEO4J_DATABASE", "SAE_502_neo4j")
query_string_destination = os.getenv("NEO4J_QUERY_STRING", "")
conn_str_destination = scheme_destination+'://'+url_destination+'/'+query_string_destination
auth_destination = 'basic_auth('+username_destination+', '+password_destination+')'

# Constatation des parametres de connexion via neo4j-driver a Neo4j 
logging.debug('NEO4J_URI : %s',url_destination)
logging.debug('NEO4J_USER : %s',username_destination)
logging.debug('NEO4J_PASSWORD : %s',password_destination)
logging.debug('NEO4J_SCHEME : %s',scheme_destination)
logging.debug('NEO4J_VERSION : %s',neo4j_version_destination)
logging.debug('NEO4J_QUERY_STRING : %s',query_string_destination)
logging.debug('NEO4J_DATABASE : %s',database_destination)
logging.debug('NEO4J_URL : %s',conn_str_destination)
logging.debug('NEO4J_AUTH : %s',auth_destination)

# Ouverture de la connexion a Neo4j
# driver = GraphDatabase.driver(conn_str_destination, auth=auth_destination)
# driver ouvre une connexion au cluster Neo4j. La base de donnees est a preciser dans le parametre database_ de driver.execute_query 
driver = GraphDatabase.driver(scheme_destination+'://'+url_destination+'/'+query_string_destination, auth=basic_auth(username_destination, password_destination))

logging.info('Connexion à Neo4j établie')

DEBUG:root:NEO4J_URI : localhost:7687
DEBUG:root:NEO4J_USER : neo4j
DEBUG:root:NEO4J_PASSWORD : password
DEBUG:root:NEO4J_SCHEME : neo4j
DEBUG:root:NEO4J_VERSION : 5
DEBUG:root:NEO4J_QUERY_STRING : 
DEBUG:root:NEO4J_DATABASE : comptoir
DEBUG:root:NEO4J_URL : neo4j://localhost:7687/
DEBUG:root:NEO4J_AUTH : basic_auth(neo4j, password)
DEBUG:neo4j.pool:[#0000]  _: <POOL> created, routing address IPv4Address(('localhost', 7687))
INFO:root:Connexion à Neo4j établie


In [23]:
def query(q: LiteralString) -> LiteralString:
    # this is a safe transform:
    # no way for cypher injection by trimming whitespace
    # hence, we can safely cast to LiteralString
    return cast(LiteralString, dedent(q).strip())

In [ ]:
def vide_graphe(database):
    """
    vide le graphe
    """
    summary = driver.execute_query(
            query("""
                MATCH (n) DETACH DELETE n
            """),
            database_=database,
            result_transformer_=neo4j.Result.consume,
    )
    nodes_deleted = summary.counters.nodes_deleted
    relationships_deleted = summary.counters.relationships_deleted
    logging.debug("delete_nodes: %i nodes deleted, delete_relationships: %i relationships deleted", nodes_deleted, relationships_deleted)

vide_graphe('comptoir')

DEBUG:neo4j:[#0000]  _: <WORKSPACE> routing towards fixed database: comptoir
DEBUG:neo4j:[#0000]  _: <WORKSPACE> pinning database: 'comptoir'
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database=AcquisitionDatabase(name='comptoir', guessed=False)
DEBUG:neo4j.pool:[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('localhost', 7687))
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> in: localhost:7687
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> dns resolver out: 127.0.0.1:7687
DEBUG:neo4j.pool:[#0000]  _: <POOL> _acquire router connection, database='comptoir', address=ResolvedIPv4Address(('127.0.0.1', 7687))
DEBUG:neo4j.pool:[#0000]  _: <POOL> trying to hand out new connection
DEBUG:neo4j.io:[#0000]  C: <OPEN> 127.0.0.1:7687
DEBUG:neo4j.io:[#C9C4]  C: <MAGIC> 0x6060B017
DEBUG:neo4j.io:[#C9C4]  C: <HANDSHAKE> 0x000001FF 0x00080805 0x00020404 0x00000003
DEBUG:neo4j.io:[#C9C4]  S: <HANDSHAKE> 0x00000005
DEBUG:neo4j.io:[#C9C4]  C: HELLO {'user_agent

In [ ]:
def initialisation_graphe(data):

    statements = [

        # --- CONTRAINTES ---

        "CREATE CONSTRAINT client_pk IF NOT EXISTS FOR (c:Client) REQUIRE c.codecli IS UNIQUE",
        "CREATE CONSTRAINT employe_pk IF NOT EXISTS FOR (e:Employe) REQUIRE e.noemp IS UNIQUE",
        "CREATE CONSTRAINT messager_pk IF NOT EXISTS FOR (m:Messager) REQUIRE m.nomess IS UNIQUE",
        "CREATE CONSTRAINT fournisseur_pk IF NOT EXISTS FOR (f:Fournisseur) REQUIRE f.nofour IS UNIQUE",
        "CREATE CONSTRAINT categorie_pk IF NOT EXISTS FOR (c:Categorie) REQUIRE c.codecateg IS UNIQUE",
        "CREATE CONSTRAINT produit_pk IF NOT EXISTS FOR (p:Produit) REQUIRE p.refprod IS UNIQUE",
        "CREATE CONSTRAINT commande_pk IF NOT EXISTS FOR (c:Commande) REQUIRE c.nocom IS UNIQUE",

        # --- INDEXS ---

        "CREATE INDEX client_societe IF NOT EXISTS FOR (c:Client) ON (c.societe)",
        "CREATE INDEX client_ville IF NOT EXISTS FOR (c:Client) ON (c.ville)",
        "CREATE INDEX client_pays IF NOT EXISTS FOR (c:Client) ON (c.pays)",

        "CREATE INDEX employe_nom IF NOT EXISTS FOR (e:Employe) ON (e.nom)",
        "CREATE INDEX employe_fonction IF NOT EXISTS FOR (e:Employe) ON (e.fonction)",
        "CREATE INDEX employe_ville IF NOT EXISTS FOR (e:Employe) ON (e.ville)",

        "CREATE INDEX produit_nom IF NOT EXISTS FOR (p:Produit) ON (p.nomProd)",
        "CREATE INDEX produit_indisponible IF NOT EXISTS FOR (p:Produit) ON (p.indisponible)",
        "CREATE INDEX produit_codeCateg IF NOT EXISTS FOR (p:Produit) ON (p.codeCateg)",

        "CREATE INDEX commande_date IF NOT EXISTS FOR (c:Commande) ON (c.dateCom)",
        "CREATE INDEX commande_codeCli IF NOT EXISTS FOR (c:Commande) ON (c.codeCli)",
        "CREATE INDEX commande_noEmp IF NOT EXISTS FOR (c:Commande) ON (c.noEmp)",

        "CREATE INDEX fournisseur_societe IF NOT EXISTS FOR (f:Fournisseur) ON (f.societe)",
        "CREATE INDEX fournisseur_ville IF NOT EXISTS FOR (f:Fournisseur) ON (f.ville)",
        "CREATE INDEX fournisseur_pays IF NOT EXISTS FOR (f:Fournisseur) ON (f.pays)",

        "CREATE INDEX categorie_nom IF NOT EXISTS FOR (c:Categorie) ON (c.nomCateg)"
    ]

    execute_statements_list(data, statements)


### Question 10 – Procédure de migration par table

Pour chaque table de la base H2GIS (CLIENT, EMPLOYE, PRODUIT, COMMANDE, DETAILCOMMANDE, etc.),
on doit définir une **procédure Python** qui :

- lit les données de la table via H2GIS (`h2gis.fetch`),
- appelle les **ordres Cypher paramétrés** correspondants pour créer les nœuds et relations dans Neo4j.

On teste ensuite le **bon fonctionnement** de chaque procédure de migration  
(nombre de nœuds/relations, cohérence des liens, absence d’erreurs).



In [ ]:
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
def ajout_table_Client(database=database_destination):
    import pandas as pd
    df_client = pd.DataFrame(h2gis.fetch("""
        SELECT codecli, societe, contact, fonction, adresse, ville, region, codepostal, pays, tel, fax
        FROM Client ORDER BY codecli
    """))

    rows_inserted = 0
    properties_inserted = 0

    for row in df_client.itertuples(index=False):
        summary = driver.execute_query(
            """
            CREATE (c:Client { 
                codecli: $codecli, societe: $societe, contact: $contact, fonction: $fonction,
                adresse: $adresse, ville: $ville, region: $region, codepostal: $codepostal,
                pays: $pays, tel: $tel, fax: $fax
            })
            """,
            database_=database,
            codecli=row.CODECLI,
            societe=row.SOCIETE,
            contact=row.CONTACT,
            fonction=row.FONCTION,
            adresse=row.ADRESSE,
            ville=row.VILLE,
            region=row.REGION,
            codepostal=row.CODEPOSTAL,
            pays=row.PAYS,
            tel=row.TEL,
            fax=row.FAX,
            result_transformer_=neo4j.Result.consume
        )
        rows_inserted += summary.counters.nodes_created
        properties_inserted += summary.counters.properties_set
    logging.getLogger("neo4j").setLevel(logging.ERROR)
    logging.getLogger("urllib3").setLevel(logging.ERROR)
    print(f"Client label: {rows_inserted} nodes inserted, {properties_inserted} properties set")



In [ ]:
ajout_table_Client() 

In [ ]:
def ajout_table_Commande(database=database_destination):
     
    df_commande = pd.DataFrame(h2gis.fetch("""
        SELECT nocom, codecli, noemp, nomess, datecom, alivavant, dateenv, destinataire, port, adrliv, villeliv, 
        regionliv, codepostalliv, paysliv 
        FROM Commande ORDER BY nocom
    """))

    rows_inserted = 0
    properties_inserted = 0

    for row in df_commande.itertuples(index=False):
        summary = driver.execute_query(
            """
            CREATE (co:Commande { 
                nocom: $nocom,
                codecli : $codecli,
                noemp: $noemp,
                nomess: $nomess,
                datecom: CASE
                    WHEN $datecom IS NULL OR $datecom = "" THEN NULL
                    ELSE date($datecom)
                END,

                aliavant: CASE
                    WHEN $alivavant IS NULL OR $alivavant = "" THEN NULL
                    ELSE date($alivavant)
                END,
                dateenv: CASE
                    WHEN $dateenv IS NULL OR $dateenv = "" THEN NULL
                    ELSE date($dateenv)
                END,
                destinataire: $destinataire, 
                port : $port,
                adrliv: $adrliv, 
                villeliv: $villeliv, 
                regionliv: $regionliv,
                codepostalliv: $codepostalliv, 
                paysliv: $paysliv
            })
            """,
            database_=database,
            nocom=row.NOCOM,
            codecli= row.CODECLI,
            noemp=row.NOEMP,
            nomess=row.NOMESS,
            datecom=None if row.DATECOM == "" else row.DATECOM,
            alivavant=None if row.ALIVAVANT == "" else row.ALIVAVANT,
            dateenv=None if row.DATEENV == "" else row.DATEENV,
            destinataire=row.DESTINATAIRE,
            port=row.PORT,
            adrliv=row.ADRLIV,
            villeliv=row.VILLELIV,
            regionliv=row.REGIONLIV,
            codepostalliv=row.CODEPOSTALLIV,
            paysliv=row.PAYSLIV,
            result_transformer_=neo4j.Result.consume,
        )
        rows_inserted += summary.counters.nodes_created
        properties_inserted += summary.counters.properties_set

    logging.info("Commande: %i nodes inserted, %i properties set",
                 rows_inserted, properties_inserted)

In [ ]:
ajout_table_Commande()

In [ ]:
def ajout_table_Produit(database=database_destination):
    
    df_results = pd.DataFrame(h2gis.fetch("""
        SELECT refprod, nofour, codecateg, nomprod, qteparunit, prixunit, unitesstock, unitescom, niveaureap, indisponible, codecateg
        FROM Produit ORDER BY refprod
    """))
    
    logging.debug(df_results)
    rows_inserted = 0
    properties_inserted = 0

    for row in df_results.itertuples(index=False):
        logging.debug(row)
        # Création du noeud Produit dans Neo4j avec paramètres
        summary = driver.execute_query(
            query("""
                CREATE (p:Produit { 
                    refprod: $refprod, nofour: $nofour, codecateg: $codecateg, nomprod: $nomprod, qteparunit: $qteparunit,
                    prixunit: $prixunit, unitesstock: $unitesstock, unitescom: $unitescom,
                    niveaureap: $niveaureap, indisponible: $indisponible
                });
            """),
            database_=database,
            refprod=row.REFPROD,
            nofour=row.NOFOUR,
            codecateg=row.CODECATEG,
            nomprod=row.NOMPROD,
            qteparunit=row.QTEPARUNIT,
            prixunit=row.PRIXUNIT,
            unitesstock=row.UNITESSTOCK,
            unitescom=row.UNITESCOM,
            niveaureap=row.NIVEAUREAP,
            indisponible=row.INDISPONIBLE,
            result_transformer_=neo4j.Result.consume,
        )
        rows_inserted += summary.counters.nodes_created
        properties_inserted += summary.counters.properties_set
        logging.debug(
            "add_produit - refprod: %i nomprod: %s result: %i nodes inserted",
            row.REFPROD, row.NOMPROD, summary.counters.nodes_created
        )

    logging.info(
        "Produit label: %i nodes inserted, %i properties set",
        rows_inserted, properties_inserted
    )


In [ ]:
ajout_table_Produit()

In [ ]:
def ajout_table_Employe(database=database_destination):
    import pandas as pd
    import logging
    from neo4j import Result
    
    # Correction : Suppression de 'chef' et utilisation de 'ADRESSE' ou 'ADDRESSE'
    # Je suppose 'ADRESSE' est correct et je le renomme en 'adresse' (lowerCamelCase) pour la variable row
    df_results = pd.DataFrame(h2gis.fetch("""
        SELECT noemp, nom, prenom, fonction, titrecourtoisie, ADRESSE AS adresse, 
               ville, region, codepostal, pays, teldom, extension, rendcomptea,
               datenaissance, dateembauche
        FROM Employe
        ORDER BY noemp
    """))

    logging.debug("Dataframe Employe récupéré :\n%s", df_results)

    rows_inserted = 0
    properties_inserted = 0

    for row in df_results.itertuples(index=False):
        logging.debug("Traitement employé : %s", row)

        # Création du noeud Employe dans Neo4j avec paramètres
        # (J'ai renommé les paramètres pour plus de clarté et de cohérence avec le modèle graphe)
        summary = driver.execute_query(
            """
            CREATE (e:Employe { 
                noemp: $noemp,
                nom: $nom,
                prenom: $prenom,
                fonction: $fonction,
                titreCourtoisie: $titreCourtoisie,
                adresse: $adresse,
                ville: $ville,
                region: $region,
                codePostal: $codePostal,
                pays: $pays,
                telDom: $telDom,
                extension: $extension,
                rendCompteA: $rendCompteA,
                datenaissance: date($datenaissance),
                dateEmbauche: date($dateEmbauche)
            });
            """,
            database_=database,
            noemp=row.NOEMP,
            nom=row.NOM,
            prenom=row.PRENOM,
            fonction=row.FONCTION,
            titreCourtoisie=row.TITRECOURTOISIE,
            adresse=row.ADRESSE,
            ville=row.VILLE,
            region=row.REGION,
            codePostal=row.CODEPOSTAL,
            pays=row.PAYS,
            telDom=row.TELDOM,
            extension=row.EXTENSION,
            rendCompteA=row.RENDCOMPTEA,
            datenaissance=row.DATENAISSANCE,
            dateEmbauche=row.DATEEMBAUCHE,
            result_transformer_=Result.consume,
        )

        rows_inserted += summary.counters.nodes_created
        properties_inserted += summary.counters.properties_set

        logging.debug(
            "add_employe - noemp: %i nom: %s, nodes created: %i",
            row.NOEMP, row.NOM, summary.counters.nodes_created
        )

    logging.info(
        "Employe label: %i nodes inserted, %i properties set",
        rows_inserted, properties_inserted
    )


In [ ]:
ajout_table_Employe()

In [ ]:
def ajout_table_Fournisseur(database=database_destination):
    
    df_results = pd.DataFrame(h2gis.fetch("""
        SELECT nofour, societe, contact, fonction, adresse, ville, region,
       codepostal, pays, tel, fax, pageaccueil
       FROM Fournisseur
    """))
    
    logging.debug(df_results)
    rows_inserted = 0
    properties_inserted = 0

    for row in df_results.itertuples(index=False):
        logging.debug(row)
        # Création du noeud Fournisseur dans Neo4j avec paramètres
        summary = driver.execute_query(
            query("""
                CREATE (f:Fournisseur { 
                    nofour: $nofour, societe: $societe, contact: $contact, fonction: $fonction,
                    adresse: $adresse, ville: $ville, region: $region, codepostal: $codepostal,
                    pays: $pays, tel: $tel, fax: $fax, pageaccueil: $pageaccueil
                });
            """),
            database_=database,
            nofour=row.NOFOUR,
            societe=row.SOCIETE,
            contact=row.CONTACT,
            fonction=row.FONCTION,
            adresse=row.ADRESSE,
            ville=row.VILLE,
            region=row.REGION,
            codepostal=row.CODEPOSTAL,
            pays=row.PAYS,
            tel=row.TEL,
            fax=row.FAX,
            pageaccueil=row.PAGEACCUEIL,
            result_transformer_=neo4j.Result.consume,
        )
        rows_inserted += summary.counters.nodes_created
        properties_inserted += summary.counters.properties_set
        logging.debug(
            "add_fournisseur - nofour: %i societe: %s result: %i nodes inserted",
            row.NOFOUR, row.SOCIETE, summary.counters.nodes_created
        )

    logging.info(
        "Fournisseur label: %i nodes inserted, %i properties set",
        rows_inserted, properties_inserted
    )


In [ ]:
ajout_table_Fournisseur()

In [ ]:
def ajout_table_Categorie(database=database_destination):
    
    df_results = pd.DataFrame(h2gis.fetch("""
        SELECT codecateg, nomcateg, description
        FROM Categorie ORDER BY codecateg
    """))
    
    logging.debug(df_results)
    rows_inserted = 0
    properties_inserted = 0

    for row in df_results.itertuples(index=False):
        logging.debug(row)
        # Création du noeud Catégorie dans Neo4j avec paramètres
        summary = driver.execute_query(
            query("""
                CREATE (c:Catégorie { 
                    codecateg: $codecateg, nomcateg: $nomcateg, description: $description
                });
            """),
            database_=database,
            codecateg=row.CODECATEG,
            nomcateg=row.NOMCATEG,
            description=row.DESCRIPTION,
            result_transformer_=neo4j.Result.consume,
        )
        rows_inserted += summary.counters.nodes_created
        properties_inserted += summary.counters.properties_set
        logging.debug(
            "add_categorie - codecateg: %i nomcateg: %s result: %i nodes inserted",
            row.CODECATEG, row.NOMCATEG, summary.counters.nodes_created
        )

    logging.info(
        "Catégorie label: %i nodes inserted, %i properties set",
        rows_inserted, properties_inserted
    )


In [ ]:
ajout_table_Categorie()

In [ ]:
def ajout_table_Messager(database=database_destination):
    
    df_results = pd.DataFrame(h2gis.fetch("""
        SELECT nomess, nommess, tel
        FROM Messager ORDER BY nomess
    """))
    
    logging.debug(df_results)
    rows_inserted = 0
    properties_inserted = 0

    for row in df_results.itertuples(index=False):
        logging.debug(row)
        # Création du noeud Messager dans Neo4j avec paramètres
        summary = driver.execute_query(
            query("""
                CREATE (m:Messager { 
                    nomess: $nomess, nommess: $nommess, tel: $tel
                });
            """),
            database_=database,
            nomess=row.NOMESS,
            nommess=row.NOMMESS,
            tel=row.TEL,
            result_transformer_=neo4j.Result.consume,
        )
        rows_inserted += summary.counters.nodes_created
        properties_inserted += summary.counters.properties_set
        logging.debug(
            "add_messager - nomess: %i nommess: %s result: %i nodes inserted",
            row.NOMESS, row.NOMMESS, summary.counters.nodes_created
        )

    logging.info(
        "Messager label: %i nodes inserted, %i properties set",
        rows_inserted, properties_inserted
    )


In [ ]:
ajout_table_Messager()

In [ ]:
def ajout_relations_simples(database=database_destination):
    """
    Migre les relations 1:N/N:1 simples (Categoriser, Fournit_Par, A_Commander, A_Realiser, A_Livrer) 
    et la relation récursive :REND_COMPTE_A.
    """
    logging.info("--- Début de la migration : Relations Simples ---")

    relationships_created = 0

    # 1. CATEGORISER (Categorie -> Produit)
    cypher_groupe = """
    MATCH (p:Produit), (c:Categorie)
    WHERE p.codecateg = c.codecateg
    CREATE (c)-[:GROUPE]->(p);
    """
    
    # 2. FOURNIT_PAR (Fournisseur -> Produit)
    cypher_fournit = """
    MATCH (p:Produit), (f:Fournisseur)
    WHERE p.nofour = f.nofour
    CREATE (f)-[:FOURNIT]->(p);
    """
    
    # 3. A_COMMANDER (Client -> Commande)
    cypher_a_passe = """
    MATCH (co:Commande), (cl:Client)
    WHERE co.codecli = cl.codecli
    CREATE (cl)-[:A_PASSE]->(co);
    """
    
    # 4. A_REALISER (Employe -> Commande)
    cypher_traite = """
    MATCH (co:Commande), (e:Employe)
    WHERE co.noemp = e.noemp
    CREATE (e)-[:TRAITE]->(co);
    """
    
    # 5. A_LIVRER (Messager -> Commande) - CORRECTION DE CASSE
    cypher_expedie = """
    MATCH (co:Commande)
    WHERE co.nomess IS NOT NULL AND co.nomess <> ''
    MATCH (m:Messager)
    WHERE m.nomess = toInteger(co.nomess)
    CREATE (m)-[:EXPEDIE]->(co);
    """
    
    # 6. REND_COMPTE_A (Employe -> Manager) - AJOUT DE LA RELATION MANQUANTE
    cypher_supervise = """
    MATCH (employe:Employe)
    MATCH (manager:Employe)
    WHERE employe.rendcomptea = manager.noemp
    CREATE (employe)-[:SUPERVISE]->(manager);
    """
    
    all_cypher = [
        cypher_groupe, cypher_fournit, cypher_a_passe, cypher_traite, cypher_expedie, 
        cypher_supervise
    ]
    
    for cypher in all_cypher:
        try:
            summary = driver.execute_query(
                query(cypher), 
                database_=database,
                result_transformer_=neo4j.Result.consume
            )
            relationships_created += summary.counters.relationships_created
        except Exception as e:
            logging.error(f"Erreur lors de la création de relation simple pour la requête : {cypher[:50]}... Détail: {e}")
            continue

        
    logging.info("Relations simples (y compris manager) : %i relations créées au total.", relationships_created)
    return relationships_created

ajout_relations_simples('comptoir')

In [ ]:
def ajout_relations_simples(database=database_destination):
    """
    Migre les relations 1:N/N:1 simples (Categoriser, Fournit_Par, A_Commander, A_Realiser, A_Livrer) 
    et la relation récursive :REND_COMPTE_A.
    """
    logging.info("--- Début de la migration : Relations Simples ---")

    relationships_created = 0

    # 1. CATEGORISER (Categorie -> Produit)
    cypher_groupe = """
    MATCH (p:Produit), (c:Categorie)
    WHERE p.codecateg = c.codecateg
    MERGE (c)-[:Groupe]->(p);
    """
    
    # 2. FOURNIT_PAR (Fournisseur -> Produit)
    cypher_fournit = """
    MATCH (p:Produit), (f:Fournisseur)
    WHERE p.nofour = f.nofour
    MERGE (f)-[:fournit_par]->(p);
    """
    
    # 3. A_COMMANDER (Client -> Commande)
    cypher_a_passe = """
    MATCH (co:Commande), (cl:Client)
    WHERE co.codecli = cl.codecli
    MERGE (cl)-[:a_Passe]->(co);
    """
    
    # 4. A_REALISER (Employe -> Commande)
    cypher_traite = """
    MATCH (co:Commande), (e:Employe)
    WHERE co.noemp = e.noemp
    MERGE (e)-[:Traite]->(co);
    """

    # 5. A_LIVRER (Messager -> Commande) - CORRECTION DE CASSE
    cypher_expedie = """
    MATCH (co:Commande)
    WHERE co.nomess IS NOT NULL AND co.nomess <> ''
    MATCH (m:Messager)
    WHERE m.nomess = toInteger(co.nomess)
    CREATE (m)-[:Expedie]->(co);
    """
    
    # 6. REND_COMPTE_A (Employe -> Manager) - AJOUT DE LA RELATION MANQUANTE
    cypher_supervise = """
    MATCH (employe:Employe)
    MATCH (manager:Employe)
    WHERE employe.rendcomptea = manager.noemp
    MERGE (employe)-[:Supervise]->(manager);
    """

    # 7. Est_Dans_Commande
    cypher_dans_commande = """
    MATCH (p:Produit)
    MATCH (co:Commande)
    WHERE p.refprod = co.nocom
    MERGE (p)-[:est_Dans_Commande]->(manager);
    """
    
    all_cypher = [
        cypher_groupe, cypher_fournit, cypher_a_passe, cypher_traite, cypher_expedie, 
        cypher_supervise
    ]
    
    for cypher in all_cypher:
        try:
            summary = driver.execute_query(
                query(cypher), 
                database_=database,
                result_transformer_=neo4j.Result.consume
            )
            relationships_created += summary.counters.relationships_created
        except Exception as e:
            logging.error(f"Erreur lors de la création de relation simple pour la requête : {cypher[:50]}... Détail: {e}")
            continue

    logging.info("Relations simples (y compris manager) : %i relations créées au total.", relationships_created)
    return relationships_created


In [ ]:
ajout_relations_simples('comptoir')

### Question 11 – Visualisation du graphe dans une page HTML autonome

On doit produire une **page HTML autonome** qui :

- affiche un **résumé du graphe** (par type de nœud et de relation),
- et, si possible, une **visualisation** (globale ou partielle) du graphe.

Cette page peut être générée depuis le calepin ou construite séparément,  
et sert à visualiser le graphe en dehors de Jupyter (par exemple pour un rendu de projet).

In [ ]:
#pip install pyvis

In [ ]:
from pyvis.network import Network


def afficher_resume_graphe(database=database_destination):
    """
    Affiche le résumé structurel du graphe (nombre de nœuds et relations par type).
    Correction: Assigne à 'records' l'élément contenant les données (le premier du retour, 
    si le driver ne renvoie pas le tuple standard (records, summary)).
    """
    logging.info(f"Récupération du résumé du graphe pour la base : {database}")
    
    cypher_resume_corrected = """
    MATCH (n) 
    RETURN head(labels(n)) AS Nom_Type, count(n) AS Nombre_Total, 'Noeud' AS Type_Entite
    UNION ALL
    MATCH ()-[r]->()
    RETURN type(r) AS Nom_Type, count(r) AS Nombre_Total, 'Relation' AS Type_Entite
    """
    
    try:
        # CORRECTION : Ne décompose plus le tuple. On assume que l'objet retourné 
        # (result) est la liste des enregistrements, ou que le premier élément est cette liste.
        result = driver.execute_query(
            cypher_resume_corrected, 
            database_=database
        )
        
        # Test de l'unpacking : si 'result' est un tuple, on prend le premier élément
        if isinstance(result, tuple) and len(result) >= 1:
            records = result[0]
        else:
            records = result
        
        print("\n--- \uD83D\uDCCB Résumé Structurel du Graphe ---")
        
        # Conversion des records en dictionnaire pour Pandas
        result_as_dicts = [dict(record) for record in records]
        df_resume = pd.DataFrame(result_as_dicts)
        display(df_resume)

    except Exception as e:
        print(f"Impossible d'obtenir le résumé. Vérifiez la connexion ou le format des données. Détail : {e}")
        

def visualiser_graphe_html(database=database_destination, output_filename="graphe_comptoir.html"):
    """
    Génère une page HTML interactive du graphe complet en extrayant les nœuds et relations.
    Correction: Assigne uniquement les enregistrements.
    """
    logging.info(f"Début de la récupération des données pour le graphe depuis la base : {database}")
    
    cypher_query = """
    MATCH (n)-[r]->(m) 
    RETURN n, r, m 
    LIMIT 2000 
    """
    cypher_isolates = """
    MATCH (n) 
    WHERE NOT (n)--() 
    RETURN n
    LIMIT 100
    """
    
    def execute_read_query_robust(query_str, database_name):
        """Fonction utilitaire pour gérer le retour non standardisé."""
        result = driver.execute_query(query_str, database_=database_name)
        # Si le résultat est un tuple (records, summary), on retourne seulement records
        if isinstance(result, tuple) and len(result) >= 1:
            return result[0]
        # Sinon, on retourne directement le résultat (qui devrait être records)
        return result

    try:
        # CORRECTION: Utilisation de la fonction robuste pour l'exécution
        records_connectes = execute_read_query_robust(cypher_query, database)
        records_isoles = execute_read_query_robust(cypher_isolates, database)
        
    except Exception as e:
        logging.error(f"Erreur lors de la récupération des données Neo4j: {e}")
        print(f"Échec de la récupération des données : {e}")
        return
        
    # --- Construction du Graphe Pyvis (Le reste du code reste inchangé) ---
    net = Network(height="750px", width="100%", notebook=False, directed=True)
    nodes_added = set()
    
    # A. Ajout des Nœuds Connectés et Relations
    for record in records_connectes:
        node_n = record['n']
        rel_r = record['r']
        node_m = record['m']
        
        n_id = node_n.element_id
        m_id = node_m.element_id
        
        n_label = list(node_n.labels)[0] if node_n.labels else "Node"
        m_label = list(node_m.labels)[0] if node_m.labels else "Node"
        r_type = rel_r.type
        
        if n_id not in nodes_added:
            net.add_node(n_id, label=f"{n_label}", title=str(dict(node_n)), group=n_label)
            nodes_added.add(n_id)
        
        if m_id not in nodes_added:
            net.add_node(m_id, label=f"{m_label}", title=str(dict(node_m)), group=m_label)
            nodes_added.add(m_id)
        
        r_properties = str(dict(rel_r))
        net.add_edge(n_id, m_id, title=r_properties, label=r_type, physics=False)

    # B. Ajout des Nœuds Isolés
    for record in records_isoles:
        node_n = record['n']
        n_id = node_n.element_id
        n_label = list(node_n.labels)[0] if node_n.labels else "Node"
        
        if n_id not in nodes_added:
            net.add_node(n_id, label=f"{n_label}", title=str(dict(node_n)), group=n_label)
            nodes_added.add(n_id)

    # --- 4. Sauvegarde du Fichier HTML ---
    net.show(output_filename, notebook=False)
    
    logging.info(f"Graphe généré avec succès dans : {output_filename}")
    print(f"Le fichier HTML interactif a été sauvegardé sous : {output_filename}. Ouvrez-le dans votre navigateur.")

# 1. Afficher le résumé structurel du graphe
afficher_resume_graphe(database=database_destination)

# 2. Générer la visualisation HTML autonome
visualiser_graphe_html(database=database_destination)


### Question 14 – Hiérarchie des employés et profondeur de supervision

On cherche à obtenir l’ensemble des **employés** ainsi que la **distance de supervision**
de ceux qui sont sous l’autorité (directe ou indirecte) du **plus haut gradé** de la base.

On produit :

- une requête SQL H2GIS qui exploite la structure hiérarchique (via `RENDCOMPTEA` / `NOEMP`),
- une requête Cypher utilisant des **chemins de longueur variable** (ex. `[:SUPERVISE*0..]`),
- un résultat incluant, pour chaque employé, le **niveau / profondeur** par rapport au supérieur hiérarchique maximal.



In [ ]:
""" 
SELECT 
    NOEMP,
    NOM,
    PRENOM,
    RENDCOMPTEA,
    0 AS PROFONDEUR
FROM EMPLOYE
WHERE RENDCOMPTEA IS NULL
UNION ALL
SELECT 
    e.NOEMP,
    e.NOM,
    e.PRENOM,
    e.RENDCOMPTEA,
    h.PROFONDEUR + 1 AS PROFONDEUR
FROM EMPLOYE e
JOIN Hierarchie h ON e.RENDCOMPTEA = h.NOEMP
)
SELECT 
    NOEMP,
    NOM,
    PRENOM,
    PROFONDEUR
FROM Hierarchie
ORDER BY PROFONDEUR, NOM, PRENOM;
"""

#Requête Cypher (Neo4j)
"!{CONNECT} MATCH (chef:Employe) WHERE NOT (:Employe)-[:SUPERVISE]->(chef) WITH chef"

"!{CONNECT} MATCH path = (chef)-[:SUPERVISE*0..]->(e:Employe) WITH e, min(length(path)) AS profondeur RETURN e.noemp AS noemp, e.nom AS nom, e.prenom AS prenom, profondeur ORDER BY profondeur, nom, prenom;"


### Question 15 – Plus court chemin entre deux employés

On doit trouver le **plus court chemin** entre deux employés précis :  
`Andrew Fuller` et `Jordan Suyama`.

On propose :

- une requête Cypher qui cherche le **plus court chemin** entre ces deux nœuds `:Employe`  
  (par exemple en utilisant `shortestPath(...)` ou des chemins avec `[:SUPERVISE*..]`),
- et on affiche la séquence de nœuds et de relations qui constitue ce chemin.

Ce résultat permet d’illustrer l’intérêt des graphes pour explorer des **chemins relationnels** entre entités.

In [ ]:
"""
WITH RECURSIVE CheminFuller AS (
    SELECT 
        NOEMP,
        NOM,
        PRENOM,
        RENDCOMPTEA,
        CAST(NOEMP AS VARCHAR(255)) AS CHEMIN,
        0 AS PROFONDEUR
    FROM EMPLOYE
    WHERE NOM = 'Fuller' AND PRENOM = 'Andrew'

    UNION ALL

    SELECT 
        e.NOEMP,
        e.NOM,
        e.PRENOM,
        e.RENDCOMPTEA,
        CAST(e.NOEMP AS VARCHAR(255)) || '->' || c.CHEMIN AS CHEMIN,
        c.PROFONDEUR + 1 AS PROFONDEUR
    FROM EMPLOYE e
    JOIN CheminFuller c 
        ON c.RENDCOMPTEA = e.NOEMP
),


CheminSuyama AS (
    SELECT 
        NOEMP,
        NOM,
        PRENOM,
        RENDCOMPTEA,
        CAST(NOEMP AS VARCHAR(255)) AS CHEMIN,
        0 AS PROFONDEUR
    FROM EMPLOYE
    WHERE NOM = 'Suyama' AND PRENOM = 'Jordan'

    UNION ALL

    SELECT 
        e.NOEMP,
        e.NOM,
        e.PRENOM,
        e.RENDCOMPTEA,
        CAST(e.NOEMP AS VARCHAR(255)) || '->' || c.CHEMIN AS CHEMIN,
        c.PROFONDEUR + 1 AS PROFONDEUR
    FROM EMPLOYE e
    JOIN CheminSuyama c 
        ON c.RENDCOMPTEA = e.NOEMP
)


SELECT 
    f.NOEMP           AS NOEMP_COMMUN,
    f.NOM             AS NOM_COMMUN,
    f.PRENOM          AS PRENOM_COMMUN,
    f.PROFONDEUR      AS PROF_FULLER,
    s.PROFONDEUR      AS PROF_SUYAMA,
    f.CHEMIN          AS CHEMIN_DEPUIS_FULLER,
    s.CHEMIN          AS CHEMIN_DEPUIS_SUYAMA,
    (f.PROFONDEUR + s.PROFONDEUR) AS DISTANCE_TOTALE
FROM CheminFuller f
JOIN CheminSuyama s 
    ON f.NOEMP = s.NOEMP
ORDER BY DISTANCE_TOTALE
FETCH FIRST 1 ROW ONLY;
"""

# Requête Cypher (Neo4j)
"!{CONNECT} MATCH (a:Employe {nom: 'Fuller',  prenom: 'Andrew'}), (b:Employe {nom: 'Suyama',  prenom: 'Jordan'}), p = shortestPath((a)-[:SUPERVISE*..]-(b)) RETURN p;"

### Gestion du projet au sein de l’équipe

Le projet a été réalisé en équipe de trois étudiantes : **Ariane Souchard**, **Rose Nerriec** et **Enora Aveline**.

La répartition du travail a été la suivante :

- **Ariane Souchard et Rose Nerriec**  
  Elles ont réalisé la plus grande partie du projet, soit environ **75 %** du travail :  
  modélisation du graphe, requêtes SQL et Cypher, mise en place de la migration H2GIS → Neo4j, rédaction et tests dans le notebook.

- **Enora Aveline**  
  Elle a contribué à hauteur d’environ **15 %** :  
  mise en forme du notebook, relectures, petites corrections et compléments sur certaines parties (modélisation / exemples).

- **Travail commun (environ 10 %)**  
  Le reste du temps a été passé **ensemble** : choix de la modélisation, discussions sur les conventions de nommage, organisation du travail et finalisation du rendu.

Cette répartition reste approximative mais reflète globalement l’implication de chacune dans le projet.

